In [1]:

api_key="Your_API_key"


***Go to This website for mistral api***
### https://console.mistral.ai/api-keys/ ###

In [ ]:
!pip install langchain-mistralai
!pip install -q -U faiss-cpu tiktoken
!pip install -qU langchain_community beautifulsoup4
!pip install langchain
!pip install sentence-transformers
!pip install langchain-chroma
!pip install -qU langchain_community pypdf

In [3]:
from langchain_community.document_loaders import WebBaseLoader

# List of URLs to load content from
urls = [
    "https://lucidmotors.com/",
    "https://lucidmotors.com/referral-program",
    "https://lucidmotors.com/gravity",
    "https://lucidmotors.com/company",
    "https://lucidmotors.com/sustainability",
    "https://lucidmotors.com/careers"# Add more URLs as needed
]

# Load content from multiple URLs
Lucid_loader = []
for url in urls:
    loader = WebBaseLoader(url)
    Lucid_loader.extend(loader.load())


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader(
#     r"C:\Users\dhira\GenAIproject\ML_GenAI_Concepts\RAG\Data\Lucid-Air-Sapphire-Owners-Manual_en-US.pdf",
#     r"C:\Users\dhira\GenAIproject\ML_GenAI_Concepts\RAG\Data\Lucid-Group-Inc-2023-Sustainability-Report.pdf"
# )
# pdf=loader.load()


In [5]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
	separator="\n\n",
	chunk_size=500,
	chunk_overlap=0,
	is_separator_regex=False,)

In [6]:
Lucid_chunks = text_splitter.split_documents(Lucid_loader)

Created a chunk of size 2818, which is longer than the specified 500
Created a chunk of size 1213, which is longer than the specified 500
Created a chunk of size 3173, which is longer than the specified 500
Created a chunk of size 1715, which is longer than the specified 500


In [7]:

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

# Set up the cache store
store = LocalFileStore("./cache/")

# Initialize the Hugging Face embedding model
core_embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Use the cache-backed embedder with the Hugging Face model
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace=core_embeddings_model.model_name
)

# Store embeddings in the Chroma vector store
vectorstore = Chroma.from_documents(Lucid_chunks, embedder, persist_directory="./chroma_store")

# Instantiate a retriever from the vector store
retriever = vectorstore.as_retriever()


C:\Users\dhira\AppData\Local\Temp\ipykernel_33260\481470647.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  core_embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\dhira\GenAIproject\ML_GenAI_Concepts\envRAG\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# this formats the docs returned by the retriever
def format_docs(docs):
	return "\n\n".join(doc.page_content for doc in docs)

# prompt to send to the LLM
prompt = """You are an assistant for question-answering tasks.
    	Use the following pieces of retrieved context to answer the question.
    	If you don't know the answer, just say that you don't know.
    	Use three sentences maximum and keep the answer concise.

    	Question: {question}

    	Context: {context}

    	Answer:
    	"""

prompt_template = ChatPromptTemplate.from_template(prompt)

# llm = ChatMistralAI(model='gpt-4o-mini', streaming=True)
llm = ChatMistralAI(
    model="mistral-large-latest", streaming=True, api_key=api_key)


# This code defines a chain where input documents are first formatted, then passed through a prompt template, and finally processed by an LLM.
rag_chain_from_docs = (
	RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
	| prompt_template
	| llm
	)
# This code creates a parallel process: one retrieves the context (using a retriever), and the other passes the question through unchanged. The results are then combined and assigned to the variable `answer` using the `rag_chain_from_docs` processing chain.
rag_chain_with_source = RunnableParallel(
	{"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [10]:

response = rag_chain_with_source.invoke("what type of cars they provide?")
print(response['answer'].content)
print(response['context'])

Lucid provides electric cars, specifically the Lucid Air (a luxury sedan) and the upcoming Lucid Gravity (an SUV). Their models include the Air Pure, Touring, Grand Touring, and Sapphire.
[Document(metadata={'description': 'Lucid is the future of sustainable mobility, designing electric cars that further reimagines the driving experience.', 'language': 'en', 'source': 'https://lucidmotors.com/', 'title': 'Home | Lucid Motors'}, page_content="Skip to main content   Lucid drives the world forward - now starting from $69,900.View Offers Lucid AirLucid GravityExperienceShopUSMenu   Accelerating the EV&nbspExperienceBUY FROM$69,900¹Air PureCHARGING ALLOWANCE$1,000on Lucid charging accessoriesLUCID CARE2 yr or 24,000&nbspmi²free scheduled maintenanceTEST DRIVEORDERPROJECTED RANGEUp to 440 mi³SEATINGUp to 7STAY INFORMEDEXPLOREMost Advanced Electric\xa0Car.Designed in California; Assembled\xa0in\xa0America;Engineered to Change the World. Current OffersExplore Current OffersLocationsExplore Loc

In [11]:
response = rag_chain_with_source.invoke("what type of lucid company")
print(response['answer'].content)
print(response['context'])

Lucid Motors is a luxury vehicle company. They launched their first product, the Lucid Air, in late 2021. The company focuses on sustainability and innovation in the automotive industry.
[Document(metadata={'description': 'There were EVs, and now there’s Lucid Motors. Learn about our commitment to the future of sustainable mobility and our leadership team.', 'language': 'en', 'source': 'https://lucidmotors.com/company', 'title': 'Company | Lucid Motors'}, page_content='Company | Lucid Motors'), Document(metadata={'description': 'Lucid is the future of sustainable mobility, designing electric cars that further reimagines the driving experience.', 'language': 'en', 'source': 'https://lucidmotors.com/', 'title': 'Home | Lucid Motors'}, page_content='Home | Lucid Motors'), Document(metadata={'description': "Lucid creates sustainable and efficient zero-emission luxury electric vehicles that make the most responsible use of the earth's resources.", 'language': 'en', 'source': 'https://lucidm

In [12]:
response = rag_chain_with_source.invoke("what is GenarativeAI?")
print(response['answer'].content)
print(response['context'])

I don't know. The provided context does not contain information about "GenerativeAI."
[Document(metadata={'description': 'The Lucid Referral Program celebrates our most passionate advocates with vehicle discounts as well as exclusive Lucid rewards and experiences.', 'language': 'en', 'source': 'https://lucidmotors.com/referral-program', 'title': 'Referral Program | Lucid Motors'}, page_content='How It Works'), Document(metadata={'description': 'The Lucid Referral Program celebrates our most passionate advocates with vehicle discounts as well as exclusive Lucid rewards and experiences.', 'language': 'en', 'source': 'https://lucidmotors.com/referral-program', 'title': 'Referral Program | Lucid Motors'}, page_content='Eligibility'), Document(metadata={'description': "Lucid creates sustainable and efficient zero-emission luxury electric vehicles that make the most responsible use of the earth's resources.", 'language': 'en', 'source': 'https://lucidmotors.com/sustainability', 'title': 'Sus

In [13]:
response = rag_chain_with_source.invoke("tell me about lucid referal programe?")
print(response['answer'].content)
print(response['context'])

The Lucid referral program allows existing customers (Referrers) to refer others (Referred Customers) using a unique link. Referrers can earn points or rewards for each qualified referral leading to a purchase or lease of a Lucid vehicle. Participation requires submission of personal information and is void where prohibited by law.
[Document(metadata={'description': 'The Lucid Referral Program celebrates our most passionate advocates with vehicle discounts as well as exclusive Lucid rewards and experiences.', 'language': 'en', 'source': 'https://lucidmotors.com/referral-program', 'title': 'Referral Program | Lucid Motors'}, page_content='Participation in the Program may require a Referred Customer and/or a Referrer to submit personal information about themselves. The personal information will be collected, processed and used in accordance with Lucid’s Privacy Policy which can be found at https://lucidmotors.com/legal. In addition, personal information may be used by Lucid on Lucid’s be